# Network Explorer

NOTE: this network viewer requires installation of `requirements.viz.txt`

In [1]:
from pathlib import Path
from network_wrangler import load_transit, load_roadway_from_dir

## Set Directories

In [2]:
EXAMPLE_DIR = Path.cwd().parent / "examples"
STPAUL = EXAMPLE_DIR / "stpaul"
SMALL = EXAMPLE_DIR / "small"

## Read in Roadway and Transit Networks

(or open them from a pkl file instead)

In [ ]:
road_net = load_roadway_from_dir(STPAUL)

In [ ]:
transit_net = load_transit(STPAUL)

In [ ]:
N_SAMPLES = 200
mymap = road_net.nodes_df.sample(N_SAMPLES).explore(
    color="grey", tiles="CartoDB positron", name="sampled original points"
)
road_net.link_shapes_df.explore(
    m=mymap, color="grey", marker_kwds={"line_opacity": 0.7}
)
mymap

In [ ]:
transit_net.shape_links_gdf.explore(
    m=mymap, color="lime green", style_kwds={"opacity": 0.5, "weight": 10}
)
transit_net.stops_gdf.explore(
    m=mymap, color="pale green", marker_kwds={"fill_opacity": 0.5, "line_opacity": 0.7}
)